In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_text")
).with_columns(
    pl.col("mluvci").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_mluvci")
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).collect()

In [4]:
vyvoj_smichu = df.explode(
    'poznamky_text'
).filter(
    pl.col('poznamky_text').str.contains('smích')
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col("text").unique().len()
).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("text") / pl.col("pocet_slov")).alias("pomer")
)

vyvoj_smichu

datum,text,pocet_slov,pomer
date,u32,u32,f64
1919-01-01,9,1646076,0.000005
1920-01-01,19,1665864,0.000011
1921-01-01,25,1996261,0.000013
1922-01-01,20,1987446,0.00001
1923-01-01,1,1245613,8.0282e-7
1924-01-01,6,1503579,0.000004
1925-01-01,5,1104633,0.000005
1926-01-01,14,1822295,0.000008
1927-01-01,8,1920843,0.000004


In [5]:
alt.Chart(
    vyvoj_smichu.to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q")
)

alt.Chart(...)

In [6]:
vyvoj_pobaveni = df.explode(
    'poznamky_text'
).filter(
    pl.col('poznamky_text').str.contains('pobavení')
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col("text").unique().len()
).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("text") / pl.col("pocet_slov")).alias("pomer")
)

vyvoj_pobaveni

datum,text,pocet_slov,pomer
date,u32,u32,f64
1994-01-01,7,948442,0.000007
1995-01-01,7,1138706,0.000006
1996-01-01,5,1241412,0.000004
1997-01-01,6,2180302,0.000003
1998-01-01,7,1523146,0.000005
1999-01-01,12,2466687,0.000005
2000-01-01,10,2486033,0.000004
2001-01-01,4,2464089,0.000002
2002-01-01,8,1933522,0.000004


In [7]:
vyvoj_veseli = df.explode(
    'poznamky_text'
).filter(
    pl.col('poznamky_text').str.contains('veselí')
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col("text").unique().len()
).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("text") / pl.col("pocet_slov")).alias("pomer")
)

vyvoj_veseli

datum,text,pocet_slov,pomer
date,u32,u32,f64
1919-01-01,1,1646076,6.0751e-7
1921-01-01,1,1996261,5.0094e-7
1947-01-01,1,1324837,7.5481e-7
1963-01-01,1,182036,0.000005
1990-01-01,1,864724,0.000001
1991-01-01,1,1433946,6.9738e-7
1993-01-01,1,1127738,8.8673e-7
1996-01-01,1,1241412,8.0553e-7
1997-01-01,1,2180302,4.5865e-7


In [8]:
alt.Chart(
    vyvoj_veseli.to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q")
)

alt.Chart(...)

In [9]:
alt.Chart(
    vyvoj_pobaveni.to_pandas(),
    width=800
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q")
)

alt.Chart(...)

In [10]:
def vyvoj_poznamky(hledani, popisek):
    
    return df.explode(
    'poznamky_text'
).filter(
    pl.col('poznamky_text').str.contains(hledani)
).group_by_dynamic(
    index_column='datum',every='1y'
).agg(
    pl.col("text").unique().len()
).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("text") / pl.col("pocet_slov")).alias("pomer")
).with_columns(
    pl.lit(popisek).alias("co")
)

In [11]:
def graf(frejm):
    return alt.Chart(
    frejm.to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q")
)
    

In [12]:
graf(vyvoj_poznamky("(?i)smích","smích"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [13]:
graf(vyvoj_poznamky("(?i)pobavení",""))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [14]:
graf(vyvoj_poznamky("(?i)veselí",""))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [15]:
graf(vyvoj_poznamky("(?i)křik","křik"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [16]:
graf(vyvoj_poznamky("(?i)pomůck","pomůcka"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [17]:
graf(vyvoj_poznamky("(?i)pláč","pláč"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [18]:
1 / vyvoj_poznamky("(?i)křik","křik").filter(pl.col("datum").dt.year().is_between(1918,1922)).select(pl.col("pomer")).mean().item()

4935.985964640689

In [19]:
1 / vyvoj_poznamky("(?i)křik","křik").filter(pl.col("datum").dt.year().is_between(2022,2025)).select(pl.col("pomer")).mean().item()

133698.17384475694

In [20]:
1 / vyvoj_poznamky("(?i)křik","křik").filter(pl.col("datum").dt.year().is_between(1918,1939)).select(pl.col("pomer")).mean().item()

3461.062838242848

In [21]:
(1 / vyvoj_poznamky("(?i)křik","křik").filter(pl.col("datum").dt.year().is_between(2022,2025)).select(pl.col("pomer")).mean().item()) / (1 / vyvoj_poznamky("(?i)křik","křik").filter(pl.col("datum").dt.year().is_between(1918,1939)).select(pl.col("pomer")).mean().item())

38.62922463223301

In [22]:
3461.062838242848 / (105*60)

0.5493750536893409

In [23]:
graf_neverbalnich_projevu = alt.Chart(
    pl.concat(
        [
            vyvoj_poznamky("(?i)kři[čk]","křik"),
            vyvoj_poznamky("(?i)hlu[kč]","hluk"),
            vyvoj_poznamky("(?i)smích","smích"),
            vyvoj_poznamky("(?i)posmě(ch|š)","posměch"),
            vyvoj_poznamky("(?i)veselí","veselí"),
            vyvoj_poznamky("(?i)pobavení","pobavení"),
            vyvoj_poznamky("(?i)hluk","hluk"),
            vyvoj_poznamky("(?i)(potlesk|tlesk[aá]|aplaus)","potlesk")
        ]
    ).to_pandas(),
    width=500,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N", sort=['potlesk','hluk','smích','posměch','křik','hluk'])
)

graf_neverbalnich_projevu

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [24]:
import os

In [25]:
try:
    os.makedirs("grafy")
except:
    pass

In [26]:
graf_neverbalnich_projevu.save(os.path.join("grafy","neverbalni-projevy.png"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


## Historické žebříčky

In [28]:
df.with_columns(pl.col("mluvci").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)smích")).group_by("jmeno").len().sort(by="len",descending=True)

jmeno,len
str,u32
"""Andrej Babiš""",424
"""Miroslav Kalousek""",342
"""Lubomír Zaorálek""",196
"""Tomio Okamura""",176
"""Zbyněk Stanjura""",132
"""David Rath""",113
"""Václav Klaus""",97
"""Ivan Langer""",82
"""Pavel Kováčik""",75


In [29]:
df.filter(pl.col("obdobi") < 1989).with_columns(pl.col("mluvci").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)smích")).group_by("jmeno").len().sort(by="len",descending=True)

jmeno,len
str,u32
"""dr. Juriga""",11
"""Post. Harus""",8
"""Jung (německy)""",6
"""Lodgman (německy)""",5
"""Posl. Harus""",4
"""Stříbrný (pokračuje)""",4
"""Juriga (pokračuje)""",4
"""Posl. Borůvka""",4
"""Radda (německy)""",4


In [30]:
df.with_columns(pl.col("mluvci").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)tlesk")).group_by("jmeno").len().sort(by="len",descending=True)

jmeno,len
str,u32
"""Miroslav Kalousek""",936
"""(uvítán potleskem)""",801
"""Vojtěch Filip""",786
"""Zbyněk Stanjura""",690
"""Alena Schillerová""",655
"""Andrej Babiš""",600
"""Lubomír Zaorálek""",574
"""Pavel Kováčik""",526
"""Bohuslav Sobotka""",523


In [31]:
df.with_columns(
    pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,poznamky_text,poznamky_mluvci,pocet_slov,jmeno
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,list[str],list[str],u32,str
"""Předseda (zvoní)""","""1918ns_ps_stenprot_001schuz_s001001.htm""",null,"""Předseda: Dr. Kramář. Zapisovatel: Člen N. S. Luděk Pik; pak větší počet členů N. S. Z kanceláře sněmovní: JUDr. Jaroslav Haasz, tajemník sněm., zástu…",1,"""ns""",1918,1,1,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,false,[],"[""zvoní""]",27,"""Předseda"""
"""Dr. Kramář (zvoní)""","""1918ns_ps_stenprot_001schuz_s001001.htm""",null,"""Slavné národní shromáždění! Z příkazu Národního výboru zahajuji Národní shromáždění. Nám všem dmou se prsa radostí, hrdostí a pýchou nade vším, co v h…",2,"""ns""",1918,1,1,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,"[""členové národního shromáždění povstávají"", ""výborně! sláva! bouřlivý potlesk."", … ""schůze opět zahájena ve 12 hodin 25 min.""]","[""zvoní""]",2229,"""Dr. Kramář"""
"""Dr. Kramář (zvoní)""","""1918ns_ps_stenprot_001schuz_s001001.htm""",null,"""Velectěné shromáždění! Odevzdáno bylo 220 lístků, znějících na jméno Tomášek (Výborně! Potlesk.) Jeden lístek byl prázdný. Prohlašuji tedy člena Národ…",3,"""ns""",1918,1,1,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,"[""výborně! potlesk."", ""výborně! potlesk."", ""poslanec tomášek vstupuje na tribunu k dru kramářovi.""]","[""zvoní""]",38,"""Dr. Kramář"""
"""Dr. Kramář""","""1918ns_ps_stenprot_001schuz_s001001.htm""",null,"""Slibujete jako člen Národního shromáždění československého a jako jeho předseda, že budete zachovávati zákony státu československého a že svůj úřad bu…",4,"""ns""",1918,1,1,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,[],[],27,"""Dr. Kramář"""
"""Předseda Tomášek (podávaje dru Kramářovi pravici)""","""1918ns_ps_stenprot_001schuz_s001001.htm""",null,"""Slibuji. (Výborně! Potlesk)""",5,"""ns""",1918,1,1,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,"[""výborně! potlesk""]","[""podávaje dru kramářovi pravici""]",2,"""Kramářovi pravici)"""
"""Předseda Tomášek (podávaje dru Kramářovi pravici)""","""1918ns_ps_stenprot_001schuz_s001002.htm""",null,"""Slavné národní shromáždění! Velectěné dámy! Vážení pánové! Jen s hlubokým vnitřním pohnutím ujímám se slova, abych prohlásil, že nesmírně čestný úkol,…",1,"""ns""",1918,1,2,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,false,"[""výborně! potlesk."", ""výborně! hlučný potlesk."", … ""opětná, bouřlivá, dlouhotrvající pochvala a potlesk.""]","[""podávaje dru kramářovi pravici""]",1220,"""Kramářovi pravici)"""
"""Posl. Pik (čte)""","""1918ns_ps_stenprot_001schuz_s001002.htm""",null,"""Slíbíte jako člen Národního shromáždění československého, že budete zachovávati zákony státu československého a že svůj úřad budete zastávati dle svéh…",2,"""ns""",1918,1,2,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,[],"[""čte""]",23,"""Posl. Pik"""
"""Předseda""","""1918ns_ps_stenprot_001schuz_s001002.htm""",null,"""Pan ředitel přečte pány podle abecedy, a žádám každého přečteného, aby povstal a prohlásil, ze slibuje.""",3,"""ns""",1918,1,2,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,[],[],15,"""Předseda"""
"""Tajemník Národ. shromáždění dr. Haas čte jména členů Národního shromáždění, vyvolaní povstávají a skládají slib slovem""","""1918ns_ps_stenprot_001schuz_s001002.htm""",null,"""Slibuji.""",4,"""ns""",1918,1,2,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,[],[],0,"""slib slovem"""


In [100]:
df.with_columns(
    pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
).explode(
    "poznamky_text"
).filter(
    pl.col(
        "poznamky_text"
    ).str.contains("(?i)kři[kč]")
).group_by(
    "jmeno"
).len(
).sort(
    by="len",descending=True
).join(
    df.with_columns(
        pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
    ).group_by("jmeno").agg(pl.col("pocet_slov").sum()),
    how="left",
    on="jmeno"
).with_columns(
    (pl.col("pocet_slov") / pl.col("len")).alias("slov_na_zakriceni")
).drop_nulls(
).filter(
    pl.col("len") > 50
).sort(
    by="slov_na_zakriceni"
)

jmeno,len,pocet_slov,slov_na_zakriceni
str,u32,u32,f64
"""vlády Udržal""",112,3848,34.357143
"""předseda Švehla""",64,6428,100.4375
"""dr Štefan""",52,6488,124.769231
"""vlády Malypetr""",103,13729,133.291262
"""dr Slávik""",154,25140,163.246753
"""Posl. Frank""",55,12484,226.981818
"""Posl. Sidor""",65,16241,249.861538
"""dr Beneš""",239,77104,322.610879
"""dr Stránský""",173,56796,328.300578


In [136]:
df.filter(pl.col("obdobi") == 2021).with_columns(
    pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
).explode(
    "poznamky_text"
).filter(
    pl.col(
        "poznamky_text"
    ).str.contains("(?i)kři[kč]")
).group_by(
    "jmeno"
).len(
).sort(
    by="len",descending=True
).join(
    df.filter(pl.col('obdobi') == 2021).with_columns(
        pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
    ).group_by("jmeno").agg(pl.col("pocet_slov").sum()),
    how="left",
    on="jmeno"
).with_columns(
    (pl.col("pocet_slov") / pl.col("len")).alias("slov_na_zakriceni")
).drop_nulls(
).sort(
    by="slov_na_zakriceni"
)

jmeno,len,pocet_slov,slov_na_zakriceni
str,u32,u32,f64
"""Jan Farský""",2,1434,717.0
"""Jan Jakob""",4,43713,10928.25
"""Josef Flek""",1,13130,13130.0
"""Jan Lipavský""",2,32219,16109.5
"""Pavel Staněk""",1,17089,17089.0
"""Zdeněk Nekula""",1,17326,17326.0
"""Josef Bělica""",1,20615,20615.0
"""Václav Král""",1,21337,21337.0
"""Jan Lacina""",1,21669,21669.0


In [32]:
df.with_columns(
    pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
).explode(
    "poznamky_text"
).filter(
    pl.col(
        "poznamky_text"
    ).str.contains("(?i)tlesk")
).group_by(
    "jmeno"
).len(
).sort(
    by="len",descending=True
).join(
    df.with_columns(
        pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
    ).group_by("jmeno").agg(pl.col("pocet_slov").sum()),
    how="left",
    on="jmeno"
).with_columns(
    (pl.col("pocet_slov") / pl.col("len")).alias("slov_na_zatleskani")
).drop_nulls(
).filter(
    pl.col("len") > 50
).sort(
    by="slov_na_zatleskani"
)

jmeno,len,pocet_slov,slov_na_zatleskani
str,u32,u32,f64
"""Posl. Slánský""",53,3674,69.320755
"""Dr. Kramář""",59,4099,69.474576
"""Posl. Smrkovský""",68,15952,234.588235
"""Posl. Frank""",52,12484,240.076923
"""Posl. Beran""",71,17316,243.887324
"""Posl. Uhlířová""",51,12597,247.0
"""(uvítána potleskem)""",58,14841,255.87931
"""Předseda Fierlinger""",149,38473,258.208054
"""Posl. Sandner""",84,22641,269.535714


In [33]:
df.with_columns(
    pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
).explode(
    "poznamky_text"
).filter(
    pl.col(
        "poznamky_text"
    ).str.contains("(?i)tlesk")
).group_by(
    "jmeno"
).len(
).sort(
    by="len",descending=True
).join(
    df.with_columns(
        pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")
    ).group_by("jmeno").agg(pl.col("pocet_slov").sum()),
    how="left",
    on="jmeno"
).with_columns(
    (pl.col("pocet_slov") / pl.col("len")).alias("slov_na_zatleskani")
).drop_nulls(
).filter(
    pl.col("len") > 50
).sort(
    by="slov_na_zatleskani"
).filter(pl.col("jmeno").str.contains("Kalousek"))

jmeno,len,pocet_slov,slov_na_zatleskani
str,u32,u32,f64
"""Miroslav Kalousek""",936,1028909,1099.261752


In [34]:
df.filter(pl.col("mluvci").str.contains("Slánský")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("tlesk")).select(pl.col("datum")).min()

datum
date
1945-11-13


In [35]:
df.filter(pl.col("mluvci").str.contains("Slánský")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("tlesk")).select(pl.col("datum")).max()

datum
date
1949-03-23


In [36]:
df.filter(pl.col("mluvci").str.contains("Dr. Kramář"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,poznamky_text,poznamky_mluvci,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,list[str],list[str],u32
"""Dr. Kramář (zvoní)""","""1918ns_ps_stenprot_001schuz_s001001.htm""",null,"""Slavné národní shromáždění! Z příkazu Národního výboru zahajuji Národní shromáždění. Nám všem dmou se prsa radostí, hrdostí a pýchou nade vším, co v h…",2,"""ns""",1918,1,1,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,"[""členové národního shromáždění povstávají"", ""výborně! sláva! bouřlivý potlesk."", … ""schůze opět zahájena ve 12 hodin 25 min.""]","[""zvoní""]",2229
"""Dr. Kramář (zvoní)""","""1918ns_ps_stenprot_001schuz_s001001.htm""",null,"""Velectěné shromáždění! Odevzdáno bylo 220 lístků, znějících na jméno Tomášek (Výborně! Potlesk.) Jeden lístek byl prázdný. Prohlašuji tedy člena Národ…",3,"""ns""",1918,1,1,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,"[""výborně! potlesk."", ""výborně! potlesk."", ""poslanec tomášek vstupuje na tribunu k dru kramářovi.""]","[""zvoní""]",38
"""Dr. Kramář""","""1918ns_ps_stenprot_001schuz_s001001.htm""",null,"""Slibujete jako člen Národního shromáždění československého a jako jeho předseda, že budete zachovávati zákony státu československého a že svůj úřad bu…",4,"""ns""",1918,1,1,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,[],[],27
"""Ministerský předseda Dr. Kramář""","""1918ns_ps_stenprot_001schuz_s001003.htm""",null,"""Přijímám. (Výborně! Hlučný potlesk.)""",4,"""ns""",1918,1,3,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,"[""výborně! hlučný potlesk.""]",[],3
"""Min. předseda Dr. Kramář (podává pravici předsedovi)""","""1918ns_ps_stenprot_001schuz_s001003.htm""",null,"""Slibuji.""",33,"""ns""",1918,1,3,1918-11-14,"""Národní shromáždění československé 1918 – 1920""","""1. schůze""",true,true,true,[],"[""podává pravici předsedovi""]",0
"""Předseda vlády Dr. Kramář""","""1918ns_ps_stenprot_002schuz_s002003.htm""",null,"""Slavné shromáždění! Já nepotřebuji vás ubezpečovati, že stejně jako naši bratři slovenští jsme my všichni do duše rozrušeni tím, co na Slovensku se dě…",6,"""ns""",1918,2,3,1918-11-15,"""Národní shromáždění československé 1918 – 1920""","""2. schůze""",true,true,true,"[""hlučný souhlas."", ""výborně! potlesk."", … ""výborně! hlučný, dlouho trvající potlesk.""]",[],1401
"""Dr. Kramář""","""1918ns_ps_stenprot_109schuz_s109007.htm""",null,"""Račte dovoliti, abych vám zase něco řekl. Hádáte se o monarchii a republiku jako o článek víry. To není článek víry, to je věc politické oportunity, n…",7,"""ns""",1918,109,7,1920-01-22,"""Národní shromáždění československé 1918 – 1920""","""109. schůze""",true,true,true,"[""hluk.""]",[],153
"""6. Za Národní demokracii s agrární oposicí, předseda Dr. Kramář""","""1920ns_ps_stenprot_001schuz_s001001.htm""",null,"""v I. Skrutiniu: Bečka Bohdan Ing., stavitel, Smíchov, Engliš Karel Dr., rektor university, Brno, Kramář Karel Dr., průmyslník, Praha, Lukavský Františ…",6,"""ps""",1920,1,1,1920-05-26,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""1. schůze""",true,true,true,[],[],119
"""Posl. Dr. Kramář (pokračuje)""","""1920ns_ps_stenprot_016schuz_s016009.htm""",null,"""Když už musím na to odpověděti, řeknu jedno: Jestliže jsme poslali agitátory na Slovensko, poslali jsme je proto, aby hájili tu myšlenku, o které zde …",4,"""ps""",1920,16,9,1920-11-04,"""Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna""","""16. schůze""",true,true,true,"[""výborně! - potlesk."", ""výkřiky.""]","[""pokračuje""]",129


In [37]:
df.filter(pl.col("obdobi") <= 1989).with_columns(pl.col("mluvci").str.replace(r"\(\w{0,100}\)","").str.strip_chars(" ").str.split(" ").list.slice(-2,2).list.join(" ").alias("jmeno")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)potlesk")).group_by("jmeno").len().sort(by="len",descending=True)

jmeno,len
str,u32
"""(uvítán potleskem)""",801
"""Předseda""",761
"""Posl. Zeminová""",266
"""(uvítaný potleskom)""",172
"""dr Beneš""",164
"""Předseda Fierlinger""",148
"""dr. Kramář""",144
"""hlučným potleskem)""",119
"""Posl. Zápotocký""",115


Fráňa Zemínová:

> Po únorovém převratu odešla na protest proti sílícímu vlivu komunistů z politického života. Přesto byla na podzim roku 1949 zatčena a následujícího roku ve svých 68 letech odsouzena k dvaceti letům vězení ve vykonstruovaném soudním procesu s Miladou Horákovou. Po jedenácti letech ve věznicích v Praze, Jihlavě a Plzni, kde pracovala jako švadlena, jí po dvou infarktech byl zbytek trestu prominut při velké amnestii prezidenta A. Novotného v roce 1960. Částečně se zachovaly její listy z vězení.

In [39]:
df.filter(pl.col("mluvci").str.contains("Zem[ií]nová")).group_by(pl.col("datum").dt.year()).len().sort(by="datum")

datum,len
i32,u32
1919,7
1920,14
1921,4
1922,6
1923,3
1924,4
1925,16
1926,5
1927,7


## Nikdo se nebude smát

In [41]:
df.filter(pl.col("datum").dt.year() <= 1955).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)smích")).sort(by="datum").tail(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,poznamky_text,poznamky_mluvci,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,list[str],u32
"""Posl. Beuer [německy]""","""1935ns_ps_stenprot_144schuz_s144001.htm""",null,"""Slavná sněmovno! To, co jsme v úterý zde ve sněmovně zažili, byl první demonstrativní krok k uskutečnění onoho komplotu fašistické iredenty bez rozdíl…",6,"""ps""",1935,144,1,1938-03-31,"""NS RČS 1935-1938, Poslanecká sněmovna""","""144. schůze""",true,true,true,"""smích komunistických poslanců.""",[],978
"""Posl. Kundt (Německy)""","""1935ns_ps_stenprot_149schuz_s149007.htm""",null,"""Dámy a pánové! Nejdříve poznámku. (Hluk a různé výkřiky.) Pan kol. Jaksch právě . . . (Hluk a různé výkřiky.) Něco takového mne nepřivede z klidu, já …",2,"""ps""",1935,149,7,1938-05-12,"""NS RČS 1935-1938, Poslanecká sněmovna""","""149. schůze""",true,true,true,"""smích poslanců strany sudetskoněmecké.""","[""německy""]",2529
"""Posl. Kundt (Německy)""","""1935ns_ps_stenprot_149schuz_s149007.htm""",null,"""Dámy a pánové! Nejdříve poznámku. (Hluk a různé výkřiky.) Pan kol. Jaksch právě . . . (Hluk a různé výkřiky.) Něco takového mne nepřivede z klidu, já …",2,"""ps""",1935,149,7,1938-05-12,"""NS RČS 1935-1938, Poslanecká sněmovna""","""149. schůze""",true,true,true,"""smích.""","[""německy""]",2529
"""Posl. Fusek""","""1945pns_stenprot_032schuz_s032007.htm""","""/sqw/detail.sqw?id=303""","""A nejinak je tomu u mléka. O mléku odstředěném jsem hovořil při kalkulaci másla. Budu hovořit o mléku egalisovaném, o mléku pro kojence a malé děti, k…",1,"""pns""",1945,32,7,1946-02-20,"""Prozatimní NS RČS 1945-1946""","""32. schůze""",true,true,false,"""hlasy: na smíchově se mluvilo v neděli jinak!""",[],2286
"""Posl. V. Beneš""","""1946uns_stenprot_056schuz_s056018.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Chci zase říci, že naši ministři na tom vinni nejsou. Ministr dr Stránský, ministr dr Dolanský, ministr Nosek jsou ochotni tyto věci udělat, ale oni t…",1,"""uns""",1946,56,18,1947-06-10,"""Ústavodárné NS RČS 1946-1948""","""56. schůze""",true,true,false,"""smích.""",[],2979


In [42]:
df.filter(pl.col("datum").dt.year() <= 1955).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)smích")).sort(by="datum").tail(1).select(pl.col("text")).item()

'Chci zase říci, že naši ministři na tom vinni nejsou. Ministr dr Stránský, ministr dr Dolanský, ministr Nosek jsou ochotni tyto věci udělat, ale oni to nemohou za ty úředníky vyřídit, napsat a expedovat, oni to nemohou udělat za úředníky v ministerstvu vnitra, kteří tuto záležitost měří ne na metry, ale na celé kilometry. Paní a pánové, chci se zmínit o § 11 retribučního dekretu, který se, žel, dotýká jedině udavačství okupantům. Tady se soudilo často smrt za smrt, a správně. Ale vedle okupantských udavačů jsou zde další zlovolní udavači našim československým autoritám, tedy ne ti, kteří udávali právem, těch se má slova netýkají, ale zlovolní udavači, kteří udávali často lidi poctivé z důvodů nikoli čestných. Na to myslil náš soudruh dr Bělehrádek, když zamýšlel doplněk k § 11 retribučního zákona, aby postihl ty lidské parasity, kteří si z udavačství vytvořili výnosný kořistnický obchod anebo nástroj své pomsty. Tito podlidé žili z lidského neštěstí. Máme zde o tom celý svazek spisů, 

In [43]:
df.filter(pl.col("datum").dt.year() >= 1955).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)smích")).sort(by="datum").head(3)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,poznamky_text,poznamky_mluvci,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,list[str],u32
"""Posl. Harus""","""1960ns_stenprot_004schuz_s004023.htm""",null,"""Vážení soudruzi a soudružky poslanci, chci se v první řadě omluvit, neboť jak jste byli svědky, přihlásil jsem se ke slovu teprve nyní, při projednává…",4,"""ns""",1960,4,23,1960-11-17,"""NS RČS 1960-1964""","""4. schůze""",true,true,true,"""smích""",[],757
"""Posl. Harus""","""1960ns_stenprot_004schuz_s004023.htm""",null,"""Vážení soudruzi a soudružky poslanci, chci se v první řadě omluvit, neboť jak jste byli svědky, přihlásil jsem se ke slovu teprve nyní, při projednává…",4,"""ns""",1960,4,23,1960-11-17,"""NS RČS 1960-1964""","""4. schůze""",true,true,true,"""smích""",[],757
"""Posl. Fogel""","""1960ns_stenprot_010schuz_s010009.htm""",null,"""Súdružky a súdruhovia poslanci! Vládny návrh zákona o disciplinárnej odpovednosti sudcov z povolania, ktorý dnes prejednávame, je jedným z mnohých opa…",6,"""ns""",1960,10,9,1961-11-29,"""NS RČS 1960-1964""","""10. schůze""",true,true,true,"""smích.""",[],983


In [44]:
df.filter(pl.col("datum").dt.year() >= 1955).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(?i)smích")).sort(by="datum").head(1).select(pl.col("text")).item()

'Vážení soudruzi a soudružky poslanci, chci se v první řadě omluvit, neboť jak jste byli svědky, přihlásil jsem se ke slovu teprve nyní, při projednávání bodu programu "Zákona o lesích". Proto se omlouvám, že by to mnozí mohli považovat za nedisciplinovanost, neboť počet řečníků k tomuto bodu byl stanoven vzhledem na čas. (Hlasy: kým?) Předsednictvem Národního shromáždění při projednávání a schvalování programu schůze. Neměl jsem ani úmysl k věci hovořit, ale během doby projednávání bodu "Zákona o lesích" tak vznikla situace, která mě nutí, abych se o slovo přihlásil a některé věci doplnil, na které zpravodaj ve své zprávě nevzpomněl a ani další řečníci se o nich nezmínili. Já je považuji za důležité. Chtěl bych při příležitosti projednávání "Zákona o lesích" zdůraznit ještě jednu důležitou národohospodářskou funkci lesa, o které, jak jsem řekl, se zpravodaj nezmínil. Tam kde je les, je i zvěř. Ta patří k lesu jako jeho součást. A není to jen okrasa lesa a ještě méně předmět lovecké vá

In [45]:
def hledej(text, n=3, delka=200, frejm=df):
    nalezy = frejm.sort(by="datum").filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [46]:
import re

In [47]:
hledej("chleba, cti a všeho",frejm=df.filter(pl.col("datum").dt.year() > 1945))

Kdo: Posl. V. Beneš
Kde: 1946uns_stenprot_056schuz_s056018.htm
Kdy: 1947-06-10
Co:  ministerstvu školství známý vlivný úředník jednoho dne s několika kamarády vyhnal bez jakéhokoliv oprávnění na 40 svých kolegů z úřadu. Jednoho z nich vsadil do kriminálu na Pankrác a ostatní zbavil chleba, cti a všeho, co s tím souvisí. A přece tento vysoký úředník nebyl čistý jako lilium. Za doby okupace učil za německé peníze, ačkoliv nebyl z němčiny zkoušen - tedy práce z pilnosti - úředníky ministerstva školst
***


In [48]:
df.filter(pl.col("text").str.contains("chleba, cti a všeho"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,poznamky_text,poznamky_mluvci,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,list[str],list[str],u32
"""Posl. V. Beneš""","""1946uns_stenprot_056schuz_s056018.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Chci zase říci, že naši ministři na tom vinni nejsou. Ministr dr Stránský, ministr dr Dolanský, ministr Nosek jsou ochotni tyto věci udělat, ale oni t…",1,"""uns""",1946,56,18,1947-06-10,"""Ústavodárné NS RČS 1946-1948""","""56. schůze""",true,true,false,"[""hlas: a sedí na ministerstvu dál!"", ""smích."", … ""bouřlivý, dlouho trvající potlesk.""]",[],2979


## Současná sněmovna

In [50]:
len(df.filter(pl.col('obdobi') == 2021).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains("křik")))

136

In [51]:
df.filter(pl.col('obdobi') == 2021).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains("křik")).group_by("mluvci").len().sort(by="len",descending=True)

mluvci,len
str,u32
"""Poslankyně Alena Schillerová""",17
"""Místopředseda PSP Jan Bartošek""",13
"""Místopředsedkyně PSP Olga Richterová""",9
"""Poslanec Tomio Okamura""",8
"""Předsedkyně PSP Markéta Pekarová Adamová""",6
"""Poslanec Jakub Michálek""",6
"""Místopředseda PSP Karel Havlíček""",5
"""Místopředseda PSP Jan Skopeček""",5
"""Poslanec Andrej Babiš""",4


In [52]:
df.filter(pl.col('obdobi') == 2021).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains("křik")).group_by("mluvci").len().sort(by="len",descending=True).filter(pl.col("mluvci").str.contains("Schil"))

mluvci,len
str,u32
"""Poslankyně Alena Schillerová""",17


In [53]:
df.filter(
    pl.col('obdobi') == 2021
).explode(
    'poznamky_text'
).filter(
    pl.col('poznamky_text').str.contains("křik")
).group_by(
    ["mluvci","datum"]
).len(
).sort(
    by="len",descending=True
)

mluvci,datum,len
str,date,u32
"""Místopředseda PSP Jan Skopeček""",2022-02-15,4
"""Místopředsedkyně PSP Olga Richterová""",2024-01-23,3
"""Místopředseda PSP Jan Bartošek""",2023-03-03,3
"""Poslanec Tomio Okamura""",2022-02-15,3
"""Poslanec Patrik Nacher""",2024-01-23,2
"""Poslankyně Alena Schillerová""",2023-03-02,2
"""Poslanec Andrej Babiš""",2024-05-28,2
"""Poslankyně Alena Schillerová""",2024-01-23,2
"""Poslanec Hubert Lang""",2023-03-03,2


In [54]:
df.filter(
    pl.col('obdobi').is_between(1918,1938)
).explode(
    'poznamky_text'
).filter(
    pl.col('poznamky_text').str.contains("křik")
).group_by(
    ["mluvci","datum"]
).len(
).sort(
    by="len",descending=True
)

mluvci,datum,len
str,date,u32
"""Posl. Šalát (pokračuje)""",1930-03-11,53
"""Ministr dr Beneš (pokračuje)""",1926-02-16,52
"""Posl. dr Štefan (pokračuje)""",1926-03-11,50
"""Posl. Gottwald (pokračuje)""",1929-12-21,48
"""Ministr dr Slávik (pokračuje)""",1931-06-02,45
"""Ministr dr Beneš (pokračuje)""",1930-05-20,44
"""Posl. Mlčoch (pokračuje)""",1933-02-02,43
"""Ministr dr Beneš (pokračuje)""",1930-01-30,42
"""Posl. Sidor (pokračuje)""",1937-12-02,40


## První republika

In [56]:
df.filter(pl.col("datum") < pl.date(1940, 12, 21)).select(pl.col("datum")).max()

datum
date
1938-12-16


In [57]:
df.filter(
    pl.col('obdobi').is_between(1918,1938)
).explode(
    'poznamky_text'
).filter(
    pl.col('poznamky_text').str.contains("křik")
).group_by(
    ["datum"]
).len(
).sort(
    by="len",descending=True
)

datum,len
date,u32
1925-12-21,164
1925-12-18,161
1928-01-25,136
1931-11-25,135
1928-01-24,129
1926-03-11,128
1922-12-15,127
1931-06-26,121
1931-02-03,119


In [58]:
df.filter(
    pl.col('obdobi').is_between(1918,1938)
).explode(
    'poznamky_text'
).filter(
    pl.col('poznamky_text').str.contains("hluk")
).group_by(
    ["datum"]
).len(
).sort(
    by="len",descending=True
)

datum,len
date,u32
1922-12-15,78
1925-12-21,78
1920-06-10,62
1925-12-18,57
1924-05-27,52
1922-11-29,51
1922-10-27,50
1921-03-11,49
1922-05-30,49


In [59]:
df.filter(pl.col("datum") == pl.date(1925, 12, 21)).select(pl.col("text")).to_series().to_list()

['Předseda: Malypetr. Místopředsedové: inž. Dostálek, Slavíček, dr Spina, Stivín. Zapisovatelé: Vávra, Zierhut. 270 poslanců podle presenční listiny. Zástupci vlády: min. předseda dr Švehla; ministři Bechyně, dr Beneš, dr Dolanský, inž. Dvořáček, dr Hodža, Mlčoch, dr Nosek, dr Srdínko, Stříbrný, Šrámek, Tučný, dr Viškovský, dr Winter; za ministerstvo spravedlnosti odb. přednosta dr Polák. Z kanceláře sněmovní: sněm. tajemník dr Říha, jeho zástupci Nebuška, dr Mikyška.',
 'Zahajuji 6. schůzi posl. sněmovny. Pan posl. dr E. Beneš dostavil se do dnešní schůze. Dosud měl dovolenou, neboť ve funkci ministra zahraničí zdržoval se v cizině, a nemohl se účastniti schůzí a složiti poslanecký slib. Ježto před tím podle §u 6 jedn. řádu v kanceláři sněmovní podepsal slibovací formuli, přikročíme ke slibu podle §u 22 úst. listiny a §u 6 jedn. řádu tím způsobem, že přečtena bude ústavou předepsaná formule slibovací jazykem českým, pan posl. dr E. Beneš ke mně přistoupí a slovem "slibuji" a podáním r

In [60]:
df.filter(pl.col("datum") == pl.date(1925, 12, 21)).explode("poznamky_text").group_by("poznamky_text").len().sort(by="len",descending=True)

poznamky_text,len
str,u32
null,63
"""výkřiky.""",21
"""výborně!""",19
"""výborně! potlesk.""",16
"""hluk.""",13
"""výkřiky komunistických poslanců.""",12
"""potlesk.""",12
"""tak je!""",10
"""potlesk komunistických poslanců.""",9


In [61]:
df.filter(pl.col("datum").dt.year().is_between(1948,1968)).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(tlesk|aplaus)"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,poznamky_text,poznamky_mluvci,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,list[str],u32
"""Ministr dr Drtina""","""1946uns_stenprot_090schuz_s090003.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Slavná sněmovno! K první otázce pana posl. Zuzaňáka, jak daleko pokročilo šetření státního zastupitelství v Olomouci ve věci atentátu na tři českoslov…",2,"""uns""",1946,90,3,1948-01-21,"""Ústavodárné NS RČS 1946-1948""","""90. schůze""",true,true,true,"""dlouhotrvající potlesk.""",[],2799
"""Posl. Hodinová-Spurná (pokračuje)""","""1946uns_stenprot_091schuz_s091002.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Soudní lidé lehce poznají, že tyto methody ministra spravedlnosti se nesou v celkové linii politiky národně socialistické strany a jejího zhoubného vl…",11,"""uns""",1946,91,2,1948-01-22,"""Ústavodárné NS RČS 1946-1948""","""91. schůze""",true,true,true,"""potlesk.""","[""pokračuje""]",178
"""Posl. dr Krajina""","""1946uns_stenprot_091schuz_s091003.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Pane předsedo, paní a pánové! Včera jsme tu slyšeli řeč ministra dr Drtiny. Dnes jsme zde slyšeli prohlášení paní posl. Hodinové-Spurné. Dávám na uváž…",1,"""uns""",1946,91,3,1948-01-22,"""Ústavodárné NS RČS 1946-1948""","""91. schůze""",true,true,true,"""potlesk.""",[],90
"""Posl. dr Krajina""","""1946uns_stenprot_091schuz_s091003.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Pane předsedo, paní a pánové! Včera jsme tu slyšeli řeč ministra dr Drtiny. Dnes jsme zde slyšeli prohlášení paní posl. Hodinové-Spurné. Dávám na uváž…",1,"""uns""",1946,91,3,1948-01-22,"""Ústavodárné NS RČS 1946-1948""","""91. schůze""",true,true,true,"""potlesk.""",[],90
"""Zpravodajka posl. Synková""","""1946uns_stenprot_092schuz_s092002.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Vážená sněmovno, paní a pánové! Soc.-politický výbor předkládá vládní návrh zákona tisk 901 o právním postavení snoubenek a nemanželských dětí, pozůst…",12,"""uns""",1946,92,2,1948-02-03,"""Ústavodárné NS RČS 1946-1948""","""92. schůze""",true,true,true,"""potlesk.""",[],521
"""Zpravodaj posl. Fusek""","""1946uns_stenprot_092schuz_s092004.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Slavná sněmovno, paní a pánové! Vládní návrh zákona o některých opatřeních v oboru rejstříkového práva sleduje v podstatě dvojí účel: jednak, aby prot…",2,"""uns""",1946,92,4,1948-02-03,"""Ústavodárné NS RČS 1946-1948""","""92. schůze""",true,true,true,"""potlesk.""",[],484
"""Místopředseda Hladký (zvoní)""","""1946uns_stenprot_093schuz_s093003.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Dávám slovo k odpovědi na tento dotaz panu ministru zemědělství v zastoupení p. ministra vnitřního obchodu. Minister Ďuriš: Pán posl. Hora sa opytuje,…",10,"""uns""",1946,93,3,1948-02-04,"""Ústavodárné NS RČS 1946-1948""","""93. schůze""",true,true,true,"""potlesk.""","[""zvoní""]",348
"""Místopředseda Hladký""","""1946uns_stenprot_093schuz_s093003.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Dávám slovo k odpovědi na tento dotaz panu ministru zemědělství. Minister Ďuriš: Pani posl. dr Horákovej odpovedám takto: Pokus samovraždy spáchal Ing…",16,"""uns""",1946,93,3,1948-02-04,"""Ústavodárné NS RČS 1946-1948""","""93. schůze""",true,true,true,"""potlesk.""",[],379
"""Posl. Jura""","""1946uns_stenprot_093schuz_s093005.htm""","""/sqw/detail.sqw?id=5723&sfo=7""","""Slavná sněmovno, paní a pánové! Smlouvu o obchodu a plavbě mezi republikou Československou a Svazem sovětských socialistických republik projednal dopr…",2,"""uns""",1946,93,5,1948-02-04,"""Ústavodárné NS RČS 1946-1948""","""93. schůze""",true,true,true,"""potlesk.""",[],532


In [62]:
df.filter(pl.col("datum").dt.year().is_between(1948,1968)).select(pl.col("pocet_slov")).sum()

pocet_slov
u32
5915016


In [63]:
5915016/5872

1007.325613079019

In [64]:
df.filter(pl.col("datum").dt.year().is_between(1969,1989)).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("(tlesk|aplaus)"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,poznamky_text,poznamky_mluvci,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,list[str],u32
"""Předseda vlády ČSSR ing. Oldřich Černík""","""1969fs_slsn_stenprot_003schuz_s003007.htm""",null,"""Vážené Federální shromáždění, soudružky a soudruzi poslanci! Vláda Československé socialistické republiky před vás předstupuje s návrhem nového progra…",3,"""fs""",1969,3,7,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,true,"""potlesk.""",[],1734
"""Posl. Ludvík Černý""","""1969fs_slsn_stenprot_003schuz_s003011.htm""",null,"""Soudruzi poslanci, s velkou pozorností jsem vyslechl vládní prohlášení přednesené s. Černíkem a pokládám jej za seriózní základ v nástupu do práce fed…",4,"""fs""",1969,3,11,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,true,"""potlesk.""",[],653
"""Posl. Mjartan""","""1969fs_slsn_stenprot_003schuz_s003011.htm""",null,"""Vážené Federálne zhromaždenie, po dlhé mesiace rozprávame o konsolidácii pomerov u nás, a nielen rozprávame, ale všestranne o konsolidáciu usilujeme. …",6,"""fs""",1969,3,11,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,true,"""potlesk.""",[],925
"""Posl. Fleyberk""","""1969fs_slsn_stenprot_003schuz_s003012.htm""",null,"""Vládní prohlášení konstatuje ve shodě s naším názorem, že důležitým znakem nové situace je obecné poznání nutnosti přejít k cílevědomé a konstruktivní…",1,"""fs""",1969,3,12,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,false,"""potlesk.""",[],242
"""Posl. dr. h. c. Plojhar""","""1969fs_slsn_stenprot_003schuz_s003012.htm""",null,"""Vážené soudružky a soudruzi poslanci, dovolte mi, abych krátce promluvil k jednomu odstavci vládního prohlášení, který je sice velmi krátký, ale který…",4,"""fs""",1969,3,12,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,true,"""potlesk.""",[],474
"""Posl. dr. h. c. Plojhar""","""1969fs_slsn_stenprot_003schuz_s003012.htm""",null,"""Vážené soudružky a soudruzi poslanci, dovolte mi, abych krátce promluvil k jednomu odstavci vládního prohlášení, který je sice velmi krátký, ale který…",4,"""fs""",1969,3,12,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,true,"""potlesk.""",[],474
"""Posl. dr. h. c. Plojhar""","""1969fs_slsn_stenprot_003schuz_s003012.htm""",null,"""Vážené soudružky a soudruzi poslanci, dovolte mi, abych krátce promluvil k jednomu odstavci vládního prohlášení, který je sice velmi krátký, ale který…",4,"""fs""",1969,3,12,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,true,"""potlesk.""",[],474
"""Posl. dr. h. c. Plojhar""","""1969fs_slsn_stenprot_003schuz_s003012.htm""",null,"""Vážené soudružky a soudruzi poslanci, dovolte mi, abych krátce promluvil k jednomu odstavci vládního prohlášení, který je sice velmi krátký, ale který…",4,"""fs""",1969,3,12,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,true,"""potlesk.""",[],474
"""Posl. prof. dr. Chytil""","""1969fs_slsn_stenprot_003schuz_s003013.htm""",null,"""Vážená sněmovno, mám čest mluvit jménem Klubu poslanců Československé strany lidové, jehož jménem chci ujistit vládu ČSSR, že spatřujeme v jejím vládn…",3,"""fs""",1969,3,13,1969-10-15,"""FS ČSSR 1969-1971""","""3. schůze""",true,true,true,"""potlesk.""",[],1151


In [65]:
df.filter(pl.col("datum").dt.year().is_between(1968,1989)).select(pl.col("pocet_slov")).sum()

pocet_slov
u32
6798131


In [66]:
(6798131 / 1007) / 105

64.29404643684684